<a href="https://colab.research.google.com/github/arthurpham/google_colab/blob/main/TARF_MC_Performance_TQF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install --upgrade tensorflow --user
!pip install tf-quant-finance
!pip install QuantLib-Python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
# reduce number of threads
os.environ['TF_NUM_INTEROP_THREADS'] = '1'
os.environ['TF_NUM_INTRAOP_THREADS'] = '1'

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tf_quant_finance as tff 
import tensorflow as tf
import functools
import pandas as pd
import time
import QuantLib as ql

In [3]:
!nvidia-smi

Thu Jun  2 18:02:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
spot = 18.0
strike = 20.0
K_lower = 15.0
K_upper = 20.0
K_knockout = 30.0
tarf_target = 5.0
step_up_ratio = 2.0

r = 0.0
volatility = 0.5

In [5]:
#@title Set up parameters

dtype = tf.float64 #@param
num_samples = 200000 #@param
num_timesteps = 53 #@param

# expiries =tf.constant( [0.0, 0.5, 1.0], dtype=dtype) # This can be a rank 1 Tensor
dt = 1. / num_timesteps
# times = [1.0]
times = tf.linspace(tf.constant(0.0, dtype=dtype), tf.constant(1.0, dtype=dtype), num_timesteps)
rate = tf.constant(r, dtype=dtype)
dividend = tf.constant(0.0, dtype=dtype)
sigma = tf.constant(volatility, dtype=dtype)
spot = tf.constant(spot, dtype=dtype)
strikes = tf.constant(strike, dtype=dtype)

def set_up_pricer(times, watch_params=False):
    """Set up European option pricing function under Black-Scholes model.
    
    Args:
        expiries: List of expiries at which to to sample the trajectories.
        watch_params: A Python bool. When `True`, gradients of the price function wrt the inputs
          are computed more efficiently. 
    Returns:
     A callable that accepts a rank 1 tensor of strikes, and scalar values for 
     the spots and  volatility values. The callable outputs prices of
     the European call options on the grid `expiries x strikes`.
    """
    def price_eu_options(strikes, spot, sigma, rate, dividend):
        # Define drift and volatility functions. 
        def drift_fn(t, x):
          del t, x
          return rate - 0.5 * sigma**2
        def vol_fn(t, x):
          del t, x
          return tf.reshape(sigma, [1, 1])
        # Use GenericItoProcess class to set up the Ito process
        process = tff.models.GenericItoProcess(
            dim=1,
            drift_fn=drift_fn,
            volatility_fn=vol_fn,
            dtype=dtype)
        log_spot = tf.math.log(tf.reduce_mean(spot))
        if watch_params:
            watch_params_list = [sigma, rate, dividend]
        else:
            watch_params_list = None
        paths = process.sample_paths(
            times=times, num_samples=num_samples,
            initial_state=log_spot, 
            watch_params=watch_params_list,
            # Select a random number generator
            random_type=tff.math.random.RandomType.SOBOL, #PSEUDO_ANTITHETIC
            time_step=dt)
        
        # tf.print(paths.shape)
        
        def tarf_payoff(element):
#             tf.print(element, summarize=-1)
            total = tf.constant(0.0, dtype=tf.float64)
            discounted_payoff = tf.constant(0.0, dtype=tf.float64)
            df = tf.constant(1.0, dtype=tf.float64)
            is_active = True
            for cur_spot in element:
                if is_active:
                    cashflow = tf.constant(0.0, dtype=tf.float64)
                    add_cashflow = False
                    if K_knockout <= cur_spot:
                        # early termination
                        is_active = False
                    if K_upper <= cur_spot: # cur_spot < K_knockout
                        cashflow = cur_spot - strike
                        add_cashflow = True
                    if cur_spot < K_lower:
                        cashflow = step_up_ratio*(cur_spot - strike)
                        add_cashflow = True

                    if add_cashflow:
                        if total + cashflow >= tarf_target:
                            cashflow = tarf_target - total
                            total += cashflow
                            discounted_payoff += df*cashflow
                            is_active = False
                        else:
                            total += cashflow
                            discounted_payoff += df*cashflow
                     
            return discounted_payoff

        reshaped_paths = tf.reshape(tf.math.exp(paths), [num_samples, num_timesteps])
        payoffs = tf.vectorized_map(tarf_payoff, reshaped_paths)
        prices = tf.reduce_mean(payoffs)

        return prices
    return price_eu_options    

price_eu_options = tf.function(set_up_pricer(times, watch_params=False),
                               input_signature=[
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64)
                               ])


In [6]:
def set_up_pricer_xla(times, watch_params=False):
    """Set up European option pricing function under Black-Scholes model.
    
    Args:
        expiries: List of expiries at which to to sample the trajectories.
        watch_params: A Python bool. When `True`, gradients of the price function wrt the inputs
          are computed more efficiently. 
    Returns:
     A callable that accepts a rank 1 tensor of strikes, and scalar values for 
     the spots and  volatility values. The callable outputs prices of
     the European call options on the grid `expiries x strikes`.
    """
    def price_eu_options(strikes, spot, sigma, rate, dividend):
        # Define drift and volatility functions. 
        def drift_fn(t, x):
          del t, x
          return rate - dividend - 0.5 * sigma**2
        def vol_fn(t, x):
          del t, x
          return tf.reshape(sigma, [1, 1])
        # Use GenericItoProcess class to set up the Ito process
        process = tff.models.GenericItoProcess(
            dim=1,
            drift_fn=drift_fn,
            volatility_fn=vol_fn,
            dtype=dtype)
        log_spot = tf.math.log(tf.reduce_mean(spot))
        if watch_params:
            watch_params_list = [sigma, rate, dividend]
        else:
            watch_params_list = None
        paths = process.sample_paths(
            times=times, num_samples=num_samples,
            initial_state=log_spot, 
            watch_params=watch_params_list,
            # Select a random number generator
            random_type=tff.math.random.RandomType.SOBOL, #PSEUDO_ANTITHETIC
            time_step=dt)
        
        def tarf_payoff(paths):
            # Shape [num_timesteps, num_samples]
            paths = tf.transpose(paths)
            cur_spot = paths[0]
            total = tf.zeros_like(cur_spot)
            discounted_payoff = tf.zeros_like(cur_spot)
            df = tf.constant(1.0, dtype=tf.float64)
            is_active = tf.ones([num_samples], dtype=tf.bool)
            i = tf.constant(0, dtype=tf.int32)
            # Explicitly define the while_loop 
            def cond(i, is_active, total, discounted_payoff):
                return i < num_timesteps

            def body(i, is_active, total, discounted_payoff):
                # Here Tensors are of shape `[num_samples]`
                cur_spot = paths[i]

                cashflow = tf.zeros_like(cur_spot)
                new_is_active = K_knockout > cur_spot
                add_cashflow = tf.where(tf.logical_or(K_upper <= cur_spot, cur_spot < K_lower),
                    True, 
                    False)

                new_cashflow = tf.where(K_upper <= cur_spot,
                    cur_spot - strike,
                    cashflow
                )
                new_cashflow = tf.where(cur_spot < K_lower,
                                    step_up_ratio*(cur_spot - strike),
                                    new_cashflow)
                new_is_active = tf.where(add_cashflow,
                    tf.where(total + new_cashflow >= tarf_target,
                            False, new_is_active),
                    new_is_active)

                new_cashflow = tf.where(add_cashflow,
                    tf.where(total + new_cashflow >= tarf_target,
                            tarf_target - total, new_cashflow),
                    new_cashflow
                    )

                new_total = tf.where(add_cashflow,
                    total + new_cashflow,
                    total
                    )
                new_discounted_payoff = tf.where(add_cashflow,
                    discounted_payoff + df * new_cashflow,
                    discounted_payoff)
                # Update values only if active
                new_cashflow = tf.where(is_active, 
                                        new_cashflow,
                                        cashflow)
                new_total = tf.where(is_active, new_total, total)
                new_discounted_payoff = tf.where(is_active, 
                                        new_discounted_payoff,
                                        discounted_payoff)
                new_is_active = tf.where(is_active, new_is_active, is_active)

                return (i + 1, new_is_active, new_total, new_discounted_payoff)
                
            _, is_active, total, discounted_payoff = tf.while_loop(
                cond, body, (i, is_active, total, discounted_payoff),
                maximum_iterations=num_timesteps,
            )
            return discounted_payoff

        reshaped_paths = tf.reshape(tf.math.exp(paths), [num_samples, num_timesteps])
        payoffs = tarf_payoff(reshaped_paths)
        prices = tf.reduce_mean(payoffs)

        return prices
    return price_eu_options

price_eu_options_xla = tf.function(set_up_pricer_xla(times, watch_params=False),
                               input_signature=[
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64)
                               ], jit_compile=True)    

In [7]:
# device = "/gpu:0"
# with tf.device(device):
#     tarf_price = price_eu_options(strikes, spot, sigma)
#     print('price', tarf_price)

In [8]:
# device = "/gpu:0"
# with tf.device(device):
#     tarf_price = price_eu_options_xla(strikes, spot, sigma)
#     print('price', tarf_price)

In [9]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_price = price_eu_options(strikes, spot, sigma, rate, dividend)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_price = price_eu_options(strikes, spot, sigma, rate, dividend)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF'.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('price', tarf_price)

------------------------
TQF gpu TARF
wall time + tracing:  20.199368000030518
options per second + tracing:  0.049506499411193915
wall time:  4.657833576202393
options per second:  0.2146920845581856
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
------------------------
TQF cpu TARF
wall time + tracing:  11.317201375961304
options per second + tracing:  0.08836106796898435
wall time:  6.734930992126465
options per second:  0.1484796208259683
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)


In [10]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_price = price_eu_options_xla(strikes, spot, sigma, rate, dividend)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_price = price_eu_options_xla(strikes, spot, sigma, rate, dividend)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF XLA'.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('price', tarf_price)

------------------------
TQF gpu TARF XLA
wall time + tracing:  2.5599894523620605
options per second + tracing:  0.39062660944845545
wall time:  0.02353215217590332
options per second:  42.495050708706096
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
------------------------
TQF cpu TARF XLA
wall time + tracing:  2.478505849838257
options per second + tracing:  0.4034688883487034
wall time:  1.9903912544250488
options per second:  0.5024137831076149
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)


In [11]:
@tf.function(jit_compile=False,
             input_signature=[tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64)
                               ])
def greeks_fn(strikes, spot, sigma, rate, dividend):
    with tf.GradientTape() as tape:
      tape.watch([spot, sigma, rate, dividend])
      prices = price_eu_options(strikes, spot, sigma, rate, dividend)
    return prices, tape.gradient(prices, [spot, sigma, rate, dividend])

@tf.function(jit_compile=True,
             input_signature=[tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64)
                               ])
def greeks_fn_xla(strikes, spot, sigma, rate, dividend):
    with tf.GradientTape() as tape:
      tape.watch([spot, sigma, rate, dividend])
      prices = price_eu_options_xla(strikes, spot, sigma, rate, dividend)
    return prices, tape.gradient(prices, [spot, sigma, rate, dividend])

In [12]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_price, tarf_greeks = greeks_fn(strikes, spot, sigma, rate, dividend)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_price, tarf_greeks = greeks_fn(strikes, spot, sigma, rate, dividend)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF price+delta+vega'.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('price', tarf_price)
        print('greeks', tarf_greeks)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/while_grad/Identity_4:0", shape=(None,), dtype=int32), values=Tensor("gradients/while_grad/Identity_3:0", shape=(None,), dtype=float64), dense_shape=Tensor("gradients/while_grad/Identity_5:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/while_grad/Identity_7:0", shape=(None,), dtype=int32), values=Tensor("gradients/while_grad/Identity_6:0", shape=(None,), dtype=float64), dense_shape=Tensor("gradients/while_grad/Identity_8:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount 

------------------------
TQF gpu TARF price+delta+vega
wall time + tracing:  10.574862480163574
options per second + tracing:  0.09456387748548119
wall time:  7.279073238372803
options per second:  0.13738012618534176
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=21.37942877385511>, <tf.Tensor: shape=(), dtype=float64, numpy=-331.9822946422317>, <tf.Tensor: shape=(), dtype=float64, numpy=217.8267631339107>, None]
------------------------
TQF cpu TARF price+delta+vega
wall time + tracing:  12.558441638946533
options per second + tracing:  0.07962771407073124
wall time:  9.509042501449585
options per second:  0.10516305925097687
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=21.37942877385511>, <tf.Tensor: shape=(), dtype=float64, numpy=-331.98229464223164>, <tf.Tensor: shape=(), dtype=float64, numpy=217.8267631

In [13]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_price, tarf_greeks = greeks_fn_xla(strikes, spot, sigma, rate, dividend)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_price, tarf_greeks = greeks_fn_xla(strikes, spot, sigma, rate, dividend)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF XLA price+delta+vega'.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('price', tarf_price)
        print('greeks', tarf_greeks)

------------------------
TQF gpu TARF XLA price+delta+vega
wall time + tracing:  3.060537576675415
options per second + tracing:  0.326739984380873
wall time:  0.07061004638671875
options per second:  14.162290653700703
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=21.37942877385511>, <tf.Tensor: shape=(), dtype=float64, numpy=-331.9822946422317>, <tf.Tensor: shape=(), dtype=float64, numpy=217.8267631339107>, <tf.Tensor: shape=(), dtype=float64, numpy=-217.8267631339107>]
------------------------
TQF cpu TARF XLA price+delta+vega
wall time + tracing:  8.380032777786255
options per second + tracing:  0.11933127548746522
wall time:  6.348258972167969
options per second:  0.15752350437879095
------------------------
price tf.Tensor(-246.927357070608, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=21.37942877385511>, <tf.Tensor: shape=(), dtype=float64, numpy=-331.98229464

In [14]:
# devices = ['gpu'] # , 'cpu'

# for device in devices:
#     with tf.device('/{}:0'.format(device)):
#         for spot in [15.0, 18.2, 20.0, 25.0]:
#             # spot = tf.constant(spot, dtype=dtype)
#             t = time.time()
#             tarf_price, tarf_greeks = greeks_fn_xla(strikes, tf.convert_to_tensor(spot, dtype=dtype), sigma, rate, dividend)
#             time_tqf0 = time.time() - t

#             t = time.time()
#             tarf_price, tarf_greeks = greeks_fn_xla(strikes, tf.convert_to_tensor(spot, dtype=dtype), sigma, rate, dividend)
#             time_tqf = time.time() - t

#             print('------------------------')
#             print('TQF {} TARF XLA price+delta+vega spot: {}'.format(device, spot))
#             print('wall time + tracing: ', time_tqf0)
#             print('options per second + tracing: ', 1.0/time_tqf0)
#             print('wall time: ', time_tqf)
#             print('options per second: ', 1.0/time_tqf)
#             print('------------------------')
#             print('price', tarf_price)
#             print('greeks', tarf_greeks)
#             # print(greeks_fn_xla.pretty_printed_concrete_signatures())            

In [15]:
devices = ['gpu'] # , 'cpu'

for device in devices:
    with tf.device('/{}:0'.format(device)):
        for tmp_spot in [tf.convert_to_tensor(x, tf.float64) for x in [15.0, 18.2, 20.0, 25.0]]:
            t = time.time()
            tarf_price, tarf_greeks = greeks_fn_xla(strikes, tmp_spot, sigma, rate, dividend)
            time_tqf0 = time.time() - t

            t = time.time()
            tarf_price, tarf_greeks = greeks_fn_xla(strikes, tmp_spot, sigma, rate, dividend)
            time_tqf = time.time() - t

            print('------------------------')
            print('TQF {} TARF XLA price+delta+vega spot: {}'.format(device, tmp_spot))
            print('wall time + tracing: ', time_tqf0)
            print('options per second + tracing: ', 1.0/time_tqf0)
            print('wall time: ', time_tqf)
            print('options per second: ', 1.0/time_tqf)
            print('------------------------')
            print('price', tarf_price)
            print('greeks', tarf_greeks)
            # print(greeks_fn_xla.pretty_printed_concrete_signatures())            

------------------------
TQF gpu TARF XLA price+delta+vega spot: 15.0
wall time + tracing:  2.4598302841186523
options per second + tracing:  0.4065321117705875
wall time:  0.06909704208374023
options per second:  14.472399539014678
------------------------
price tf.Tensor(-489.18585120429816, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=47.21793991971346>, <tf.Tensor: shape=(), dtype=float64, numpy=-375.1297930074455>, <tf.Tensor: shape=(), dtype=float64, numpy=358.638751028938>, <tf.Tensor: shape=(), dtype=float64, numpy=-358.638751028938>]
------------------------
TQF gpu TARF XLA price+delta+vega spot: 18.2
wall time + tracing:  2.855764150619507
options per second + tracing:  0.35016897308661427
wall time:  0.0693674087524414
options per second:  14.41599186108858
------------------------
price tf.Tensor(-234.51510001171167, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=20.189230768587276>, <tf.Tensor: shape=(), dtype=fl

In [16]:
print(greeks_fn_xla.pretty_printed_concrete_signatures())  

greeks_fn_xla(strikes, spot, sigma, rate, dividend)
  Args:
    strikes: float64 Tensor, shape=()
    spot: float64 Tensor, shape=()
    sigma: float64 Tensor, shape=()
    rate: float64 Tensor, shape=()
    dividend: float64 Tensor, shape=()
  Returns:
    (<1>, [<2>, <3>, <4>, <5>])
      <1>: float64 Tensor, shape=()
      <2>: float64 Tensor, shape=()
      <3>: float64 Tensor, shape=()
      <4>: float64 Tensor, shape=()
      <5>: float64 Tensor, shape=()


In [17]:
# import numpy as np
# import tensorflow.compat.v2 as tf
# import tf_quant_finance as tff

dtype = tf.float64
dim = 1
valuation_date = [(2020, 1, 1)]
lv_year = dim * [[2021, 2022]]
lv_month = dim * [[1, 1]]
lv_day = dim * [[1, 1]]
lv_expiries = tff.datetime.dates_from_year_month_day(lv_year, lv_month, lv_day)
lv_strikes = dim * [[[0.1, 0.9, 1.0, 1.1, 3], [0.1, 0.9, 1.0, 1.1, 3]]]
lv_iv = dim * [[[0.5, 0.5, 0.5, 0.5, 0.5], [0.55, 0.5, 0.5, 0.5, 0.5]]]
lv_spot = dim * [1.0]
df = lambda t: tf.math.exp(-tf.convert_to_tensor([0.02], dtype=dtype) * t)

# lv = tff.experimental.local_volatility.LocalVolatilityModel.from_market_data(
#     dim, valuation_date, expiries, strikes, iv, spot, df, [0.0], dtype=dtype)
lv = tff.experimental.local_volatility.LocalVolatilityModel.from_market_data(
                1, valuation_date, lv_expiries, lv_strikes, lv_iv, lv_spot, df, [0.0], dtype=dtype)

# num_samples = 10000
paths = lv.sample_paths(
    [1.0, 1.5, 2.0],
    num_samples=10000,
    initial_state=spot,
    time_step=0.1,
    random_type=tff.math.random.RandomType.STATELESS_ANTITHETIC,
    seed=[1, 2])
# paths.shape = (10000, 3, 2)
print(paths.shape)
print(paths)

(10000, 3, 1)
tf.Tensor(
[[[14.09722114]
  [16.19684551]
  [13.29868831]]

 [[12.5832947 ]
  [10.38507071]
  [ 8.79123746]]

 [[18.74618597]
  [14.64058388]
  [13.05219019]]

 ...

 [[39.46802971]
  [45.22294174]
  [72.28176199]]

 [[29.10263318]
  [33.05029557]
  [36.47602767]]

 [[19.70081056]
  [26.52652951]
  [12.79171514]]], shape=(10000, 3, 1), dtype=float64)


In [18]:
# num_samples = 20000

valuation_date = [(2020, 1, 1)]
lv_year = [[2020, 2021]]
lv_month = [[6, 1]]
lv_day = [[1, 1]]
lv_expiries = tff.datetime.dates_from_year_month_day(lv_year, lv_month, lv_day)
lv_strikes = [[[0.1, 0.9, 1.0, 1.1, 3], [0.1, 0.9, 1.0, 1.1, 3]]]
lv_iv = [[[0.5, 0.5, 0.5, 0.5, 0.5], [0.55, 0.5, 0.5, 0.5, 0.5]]]

def set_up_pricer_xla_localvol(times, watch_params=False):
    """Set up European option pricing function under Black-Scholes model.
    
    Args:
        expiries: List of expiries at which to to sample the trajectories.
        watch_params: A Python bool. When `True`, gradients of the price function wrt the inputs
          are computed more efficiently. 
    Returns:
     A callable that accepts a rank 1 tensor of strikes, and scalar values for 
     the spots and  volatility values. The callable outputs prices of
     the European call options on the grid `expiries x strikes`.
    """
    def price_eu_options(strikes, spot, sigma, rate, dividend):
        # Define drift and volatility functions. 
        # def drift_fn(t, x):
        #   del t, x
        #   return rate - dividend - 0.5 * sigma**2
        # def vol_fn(t, x):
        #   del t, x
        #   return tf.reshape(sigma, [1, 1])
        # Use GenericItoProcess class to set up the Ito process
        # process = tff.models.GenericItoProcess(
        #     dim=1,
        #     drift_fn=drift_fn,
        #     volatility_fn=vol_fn,
        #     dtype=dtype)
        
        df = lambda t: tf.math.exp(-rate * t)
        process = tff.experimental.local_volatility.LocalVolatilityModel.from_market_data(
                1, valuation_date, lv_expiries, lv_strikes, lv_iv, spot, df, [0.0], dtype=dtype)


        # log_spot = tf.math.log(tf.reduce_mean(spot))
        if watch_params:
            watch_params_list = [sigma, rate, dividend]
        else:
            watch_params_list = None
        paths = process.sample_paths(
            times=times, num_samples=num_samples,
            initial_state=spot, 
            watch_params=watch_params_list,
            # Select a random number generator
            random_type=tff.math.random.RandomType.SOBOL, #PSEUDO_ANTITHETIC
            time_step=dt)
        
        # tf.print(paths.shape)
        
        def tarf_payoff(paths):
            # Shape [num_timesteps, num_samples]
            paths = tf.transpose(paths)
            cur_spot = paths[0]
            total = tf.zeros_like(cur_spot)
            discounted_payoff = tf.zeros_like(cur_spot)
            df = tf.constant(1.0, dtype=tf.float64)
            is_active = tf.ones([num_samples], dtype=tf.bool)
            i = tf.constant(0, dtype=tf.int32)
            # Explicitly define the while_loop 
            def cond(i, is_active, total, discounted_payoff):
                return i < num_timesteps

            def body(i, is_active, total, discounted_payoff):
                # Here Tensors are of shape `[num_samples]`
                cur_spot = paths[i]

                cashflow = tf.zeros_like(cur_spot)
                new_is_active = K_knockout > cur_spot
                add_cashflow = tf.where(tf.logical_or(K_upper <= cur_spot, cur_spot < K_lower),
                    True, 
                    False)

                new_cashflow = tf.where(K_upper <= cur_spot,
                    cur_spot - strike,
                    cashflow
                )
                new_cashflow = tf.where(cur_spot < K_lower,
                                    step_up_ratio*(cur_spot - strike),
                                    new_cashflow)
                new_is_active = tf.where(add_cashflow,
                    tf.where(total + new_cashflow >= tarf_target,
                            False, new_is_active),
                    new_is_active)

                new_cashflow = tf.where(add_cashflow,
                    tf.where(total + new_cashflow >= tarf_target,
                            tarf_target - total, new_cashflow),
                    new_cashflow
                    )

                new_total = tf.where(add_cashflow,
                    total + new_cashflow,
                    total
                    )
                new_discounted_payoff = tf.where(add_cashflow,
                    discounted_payoff + df * new_cashflow,
                    discounted_payoff)
                # Update values only if active
                new_cashflow = tf.where(is_active, 
                                        new_cashflow,
                                        cashflow)
                new_total = tf.where(is_active, new_total, total)
                new_discounted_payoff = tf.where(is_active, 
                                        new_discounted_payoff,
                                        discounted_payoff)
                new_is_active = tf.where(is_active, new_is_active, is_active)

                return (i + 1, new_is_active, new_total, new_discounted_payoff)
                
            _, is_active, total, discounted_payoff = tf.while_loop(
                cond, body, (i, is_active, total, discounted_payoff),
                maximum_iterations=num_timesteps,
            )
            return discounted_payoff

        reshaped_paths = tf.reshape(paths, [num_samples, num_timesteps])
        payoffs = tarf_payoff(reshaped_paths)
        prices = tf.reduce_mean(payoffs)

        return prices
    return price_eu_options

price_eu_options_localvol = set_up_pricer_xla_localvol(times, watch_params=True)   

price_eu_options_xla_localvol = tf.function(set_up_pricer_xla_localvol(times, watch_params=True),
                               input_signature=[
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64),
                                                tf.TensorSpec([], dtype=tf.float64)
                               ], jit_compile=True)

@tf.function(jit_compile=True,
             input_signature=[tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64),
                            tf.TensorSpec([], dtype=tf.float64)
                               ])
def greeks_fn_xla_localvol(strikes, spot, sigma, rate, dividend):
    with tf.GradientTape() as tape:
      tape.watch([spot, sigma, rate, dividend])
      prices = price_eu_options_localvol(strikes, spot, sigma, rate, dividend)
    return prices, tape.gradient(prices, [spot, sigma, rate, dividend])

# print(price_eu_options_xla(strikes, spot, sigma, rate, dividend))
# print(price_eu_options_xla_localvol(strikes, spot, sigma, rate, dividend))
# greeks_fn_xla_localvol(strikes, spot, sigma, rate, dividend)    

In [19]:
# print(price_eu_options_xla_localvol(strikes, spot, sigma, rate, dividend))
# print(greeks_fn_xla_localvol(strikes, spot, sigma, rate, dividend)) 

In [20]:
# @tf.function(jit_compile=True,
#              input_signature=[tf.TensorSpec([], dtype=tf.float64),
#                             tf.TensorSpec([], dtype=tf.float64),
#                             tf.TensorSpec([], dtype=tf.float64),
#                             tf.TensorSpec([], dtype=tf.float64),
#                             tf.TensorSpec([], dtype=tf.float64)
#                                ])
# def delta_fn(strikes, spot, sigma, rate, dividend):
#     fn = lambda spot: price_eu_options_localvol(strikes, spot, sigma, rate, dividend)
#     return tff.math.fwd_gradient(fn, spot,
#                                  use_gradient_tape=True)
    
# delta_fn(strikes, spot, sigma, rate, dividend)    

In [21]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_price = price_eu_options_xla_localvol(strikes, spot, sigma, rate, dividend)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_price = price_eu_options_xla_localvol(strikes, spot, sigma, rate, dividend)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF XLA LocalVol Price '.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('price', tarf_price)

------------------------
TQF gpu TARF XLA LocalVol Price 
wall time + tracing:  70.71524453163147
options per second + tracing:  0.014141222400110521
wall time:  0.7479372024536133
options per second:  1.3370106430319189
------------------------
price tf.Tensor(-246.9273570885836, shape=(), dtype=float64)
------------------------
TQF cpu TARF XLA LocalVol Price 
wall time + tracing:  154.13601851463318
options per second + tracing:  0.006487776248775124
wall time:  111.0559253692627
options per second:  0.009004472266337741
------------------------
price tf.Tensor(-246.9273570885836, shape=(), dtype=float64)


In [22]:
devices = ['gpu', 'cpu']

for device in devices:
    with tf.device('/{}:0'.format(device)):
        t = time.time()
        tarf_price, tarf_greeks = greeks_fn_xla_localvol(strikes, spot, sigma, rate, dividend)
        time_tqf0 = time.time() - t

        t = time.time()
        tarf_price, tarf_greeks = greeks_fn_xla_localvol(strikes, spot, sigma, rate, dividend)
        time_tqf = time.time() - t

        print('------------------------')
        print('TQF {} TARF XLA LocalVol price+delta+vega'.format(device))
        print('wall time + tracing: ', time_tqf0)
        print('options per second + tracing: ', 1.0/time_tqf0)
        print('wall time: ', time_tqf)
        print('options per second: ', 1.0/time_tqf)
        print('------------------------')
        print('price', tarf_price)
        print('greeks', tarf_greeks)

------------------------
TQF gpu TARF XLA LocalVol price+delta+vega
wall time + tracing:  64.79481410980225
options per second + tracing:  0.015433333882328688
wall time:  0.7736227512359619
options per second:  1.2926196888630426
------------------------
price tf.Tensor(-246.9273570885836, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, numpy=nan>]
------------------------
TQF cpu TARF XLA LocalVol price+delta+vega
wall time + tracing:  148.57438135147095
options per second + tracing:  0.006730635462882239
wall time:  111.92326974868774
options per second:  0.008934692510729876
------------------------
price tf.Tensor(-246.9273570885836, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.

In [23]:
devices = ['gpu'] # , 'cpu'

for device in devices:
    with tf.device('/{}:0'.format(device)):
        for tmp_spot in [tf.convert_to_tensor(x, tf.float64) for x in [15.0, 18.2, 20.0, 25.0]]:
            t = time.time()
            tarf_price, tarf_greeks = greeks_fn_xla_localvol(strikes, tmp_spot, sigma, rate, dividend)
            time_tqf0 = time.time() - t

            t = time.time()
            tarf_price, tarf_greeks = greeks_fn_xla_localvol(strikes, tmp_spot, sigma, rate, dividend)
            time_tqf = time.time() - t

            print('------------------------')
            print('TQF {} TARF XLA LocalVol price+delta+vega spot: {}'.format(device, tmp_spot))
            print('wall time + tracing: ', time_tqf0)
            print('options per second + tracing: ', 1.0/time_tqf0)
            print('wall time: ', time_tqf)
            print('options per second: ', 1.0/time_tqf)
            print('------------------------')
            print('price', tarf_price)
            print('greeks', tarf_greeks)
            # print(greeks_fn_xla.pretty_printed_concrete_signatures())            

------------------------
TQF gpu TARF XLA LocalVol price+delta+vega spot: 15.0
wall time + tracing:  0.7750897407531738
options per second + tracing:  1.2901731856601215
wall time:  0.7734165191650391
options per second:  1.292964366832473
------------------------
price tf.Tensor(-489.1858512606903, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, numpy=nan>]
------------------------
TQF gpu TARF XLA LocalVol price+delta+vega spot: 18.2
wall time + tracing:  0.7729067802429199
options per second + tracing:  1.2938170883760471
wall time:  0.7731106281280518
options per second:  1.293475944602288
------------------------
price tf.Tensor(-234.51510001478576, shape=(), dtype=float64)
greeks [<tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, numpy=nan>, <tf.Tensor: shape=(), dtype=float64, 